In [1]:
import sys
sys.path.append("..")
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter


In [2]:
import utils.dataset as myDataset
import utils.loss as myLoss
import model.model as myModel


In [3]:
arg_batchSize = 72
arg_nEpoch = 100
arg_pretrainedModel = None
# arg_pretrainedModel = "../model/pretrainedModel/final_facedet.pt"
arg_workers = 12
arg_dataset = "../data/"
arg_split = "train"
arg_outName = "facedet.pt"


In [4]:
dataset = myDataset.FaceDataset(datapath = arg_dataset, split = arg_split)
dataloader = torch.utils.data.DataLoader(dataset, shuffle = True, batch_size = arg_batchSize, \
                                         num_workers = arg_workers, drop_last = False)


1001it [00:23, 42.65it/s]


In [7]:
writer = SummaryWriter("../log/scene")

print("length of dataset: %s" % (len(dataloader)))
batch_num = len(dataloader)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = myModel.FaceKeypointModel()
model.apply(myModel.weights_init)

if arg_pretrainedModel != None:
    model.load_state_dict(torch.load("../model/" + arg_pretrainedModel))
    print("Use model from ../model/" + arg_pretrainedModel)
else:
    print("Use new model")

if not os.path.exists("../model/pretrainedModel"):
    os.makedirs("../model/pretrainedModel")

model.cuda()
model.train()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (0.9, 0.999))
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.2)
criterion = nn.MSELoss()
loss_sum = 0

length of dataset: 14
Use new model


In [8]:
for epoch in tqdm(range(arg_nEpoch)):
    for i, data in tqdm(enumerate(dataloader)):

        image, anno, gtmap= data
        image, anno, gtmap= image.to(device,  dtype=torch.float), anno.to(device), gtmap.to(device, dtype=torch.float)
        image = image/255.0
        
        heatMap = model(image)
        
        # loss = myLoss.calLossMSE(heatMap, anno, gtmap)
        loss = criterion(heatMap, gtmap)
        
        loss.backward()
        optimizer.step()

        writer.add_scalar("training loss", loss.item(), epoch*len(dataloader) + i)

        loss_sum = loss_sum + loss.item()

    print("[ epoch: %d/%d  batch: %d/%d ]  loss: %f" % (epoch, arg_nEpoch, i + 1, batch_num, loss_sum))
    loss_sum = 0
    if epoch % 5 == 4:
        torch.save(model.state_dict(), "../model/pretrainedModel/epo" + str(epoch) + arg_outName)
        print("Model saved at ../model/pretrainedModel/epo" + str(epoch) + arg_outName)


14it [00:11,  1.21it/s]
  1%|          | 1/100 [00:11<19:36, 11.89s/it]

[ epoch: 0/100  batch: 14/14 ]  loss: 3.388014


14it [00:11,  1.18it/s]
  2%|▏         | 2/100 [00:24<19:39, 12.04s/it]

[ epoch: 1/100  batch: 14/14 ]  loss: 3.271767


14it [00:12,  1.11it/s]
  3%|▎         | 3/100 [00:36<20:04, 12.42s/it]

...

[ epoch: 97/100  batch: 14/14 ]  loss: 0.148767


14it [00:12,  1.16it/s]
 99%|█████████▉| 99/100 [20:42<00:12, 12.41s/it]

[ epoch: 98/100  batch: 14/14 ]  loss: 0.148828


14it [00:12,  1.15it/s]
100%|██████████| 100/100 [20:55<00:00, 12.55s/it]

[ epoch: 99/100  batch: 14/14 ]  loss: 0.148897
Model saved at ../model/pretrainedModel/epo99facedet.pt


In [ ]:
torch.save(model.state_dict(), "../model/pretrainedModel/1000_target_" + arg_outName)
print("Model saved at ../model/pretrainedModel/final_" + arg_outName)